# The key concept to master LCEL (runnable execution order)

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
MODEL_GPT = 'gpt-4o-mini'

In [3]:
from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-3.5-turbo-0125")
model = ChatOpenAI(model=MODEL_GPT)

## Legacy Chain vs. LCEL Chain

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

### Legacy Chain

In [5]:
from langchain.chains import LLMChain

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

traditional_chain = LLMChain(
    llm=model,
    prompt=prompt
)

traditional_chain.predict(soccer_player="Maradona")

C:\Users\Pavel\AppData\Local\Temp\ipykernel_7788\3010822335.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  traditional_chain = LLMChain(


'One curious fact about Diego Maradona is that he once had a pet dog named "Diego" that was gifted to him by his friend, the famous Argentine singer and songwriter, Andrés Calamaro. Maradona loved the dog so much that he would often take him with him when he traveled, and he even referred to the dog as his "best friend." This reflects Maradona\'s deep affection for animals and his more personal side, contrasting with his larger-than-life persona on the football field.'

### New LCEL Chain

In [6]:
chain = prompt | model | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

'One curious fact about Cristiano Ronaldo is that he has a unique love for the sport that goes beyond just playing. He is known to have a very superstitious routine before matches, which includes wearing specific items or performing certain rituals. For example, he always puts on his right sock before his left and has a specific way of tying his shoelaces. These rituals help him feel more prepared and focused before taking to the field. This dedication to superstition is common among many athletes, but Ronaldo’s meticulous approach has become a notable part of his pre-game routine.'

## Let's see this graphically

![alt text](lcel-2.png)


## Replicate this process without using the LCEL chain

### First, we apply .invoke() with user input to prompt template

In [7]:
prompt.invoke({"soccer_player": "Ronaldo"})

ChatPromptValue(messages=[HumanMessage(content='tell me a curious fact about Ronaldo', additional_kwargs={}, response_metadata={})])

### Then, with completed prompt, we apply .invoke() to model

In [8]:
from langchain_core.messages.human import HumanMessage

output_after_first_step = [HumanMessage(content='tell me a curious fact about Ronaldo')]

In [9]:
model.invoke(output_after_first_step)

AIMessage(content="One curious fact about Cristiano Ronaldo is that he has a unique way of maintaining his physical fitness: he has a personal chef who prepares all his meals. Ronaldo's diet focuses on lean proteins, whole grains, fruits, and vegetables, and he reportedly eats around six small meals a day. This meticulous approach to nutrition is a key part of his training regimen and has contributed to his longevity and success in professional football. Additionally, he is known for avoiding sugary drinks and prefers water and protein shakes to stay hydrated and energized.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 14, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737

### And finally, we apply .invoke() to output parser with output of model

In [10]:
from langchain_core.messages.ai import AIMessage

output_after_second_step = AIMessage(content='One curious fact about Cristiano Ronaldo is that he does not have any tattoos on his body. Despite the fact that many professional athletes have tattoos, Ronaldo has chosen to keep his body ink-free.', response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 14, 'total_tokens': 52}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c9812511-043a-458a-bfb8-005bc0d057fb-0', usage_metadata={'input_tokens': 14, 'output_tokens': 38, 'total_tokens': 52})

In [11]:
output_parser.invoke(output_after_second_step)

'One curious fact about Cristiano Ronaldo is that he does not have any tattoos on his body. Despite the fact that many professional athletes have tattoos, Ronaldo has chosen to keep his body ink-free.'

## Ways to execute Runnables

#### LCEL Chains/Runnables are used with:
* chain.invoke(): call chain on input
* chain.stream(): call chain on input and stream back chunks of response
* chain.batch(): call chain on list of inputs

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("Tell me one sentence about {politician}.")
chain = prompt | model

In [13]:
chain.invoke({"politician": "Churchill"})

AIMessage(content='Winston Churchill was a British statesman, military leader, and writer, best known for his role as Prime Minister during World War II, where his resolute leadership and stirring speeches inspired a nation to persevere against Nazi Germany.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 14, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-e87204f8-8bcb-47d9-a27c-f635e7cc7a7a-0', usage_metadata={'input_tokens': 14, 'output_tokens': 47, 'total_tokens': 61, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [14]:
for s in chain.stream({"politician": "F.D. Roosevelt"}):
    print(s.content, end="", flush=True)

Franklin D. Roosevelt was the 32nd President of the United States, serving from 1933 to 1945, and is best known for his leadership during the Great Depression and World War II.

In [15]:
chain.batch([{"politician": "Lenin"}, {"politician": "Stalin"}])

[AIMessage(content='Vladimir Lenin was a revolutionary leader and the architect of the Soviet state, known for his role in the Bolshevik Revolution of 1917 and the establishment of a one-party socialist system in Russia.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 14, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-cf1c6efd-b912-449b-bd54-6adce5c8dcc5-0', usage_metadata={'input_tokens': 14, 'output_tokens': 42, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='Joseph Stalin was the leader of the Soviet Union from 

#### LCEL Chains/Runnables can be also used asynchronously:
* chain.ainvoke(): call chain on input
* chain.astream(): call chain on input and stream back chunks of response
* chain.abatch(): call chain on list of inputs